# Asymmetry `n_m_bg_start=2`

In [ ]:
cd ..

In [ ]:
import lightkurve as lk
from matplotlib import pyplot as  plt
import numpy as np
from astropy.table import Table
from analysis_tools_cython import *
import scipy.integrate as integrate
import numpy as np
from scipy.signal import savgol_filter

plt.rcParams["figure.figsize"] = (10, 6)

## Functions

In [ ]:
def skewed_gaussian_curve_fit(x,y):
    # Initial parameters guess
    ## i = index of min time
    ## x = time
    ## y = flux
    
    i = np.argmin(y)
    width = x[-1]-x[0]
    ### params initialisation for skewness, time, mean and sigma
    params_init = [0.1,x[i],0.1,0.0001] # i find these good to initialise with
    
    params_bounds = [[-np.inf,x[0],0,0], [np.inf,x[-1],width/2,width/2]]
    
    A0 = y[i]
    mu0 = x[i]
    sigma0 = (x[-1]-x[0])/4

    params_bounds = [[0.1,x[0],0,0], [10,x[-1],np.inf,sigma0*4]]
    
    params,cov = curve_fit(skewed_gaussian,x,y,p0=params_init,bounds=params_bounds,maxfev=1000000)
    return params, cov 

def skewed_gaussian(x,a,mean,sigma,m):
    """
    m: amplitude
    x: time
    a: skewness
    mean: time
    sigma: sigma/standard deviation    
    
    """
    return -m * skewnorm.pdf(x,a,loc=mean,scale=sigma)

def lookup_tic(tic):
    return lookup[lookup.TIC_ID == tic]

def search_method(data):
    try:
        data = data['time','corrected flux','quality','flux error']
    except:
        data = data['TIME','PDCSAP_FLUX','QUALITY','PDCSAP_FLUX_ERR']

    flat_flux, trend_flux = smoothing(data,method='median')
    a = Table()
    a['time'] = data[data.colnames[0]]
    a['flux'] = flat_flux - np.ones(len(flat_flux))
    a['quality'] = data[data.colnames[2]]
    a['flux_error'] = data[data.colnames[3]]
    #table[table.colnames[1]] = flat_flux - np.ones(len(flat_flux)) # resets normalisation to zero instead of one.
    t, flux, quality, real, flux_error = clean_data(a)
    flux *= real
    timestep = calculate_timestep(data)
    factor = ((1/48)/timestep)
    try:
        flux_error = normalise_error(data['corrected flux'],a['flux_error'])
    except:
        flux_error = normalise_error(data['PDCSAP_FLUX'],a['flux_error'])
    T1 = test_statistic_array(flux,60 * factor)

    m, n = np.unravel_index(
    T1.argmin(), T1.shape
    )  # T.argmin(): location of  T.shape: 2D array with x,y points in that dimension
    minT = T1[m, n]
    #Ts = nonzero(T1[m,n]).std()
    minT_time = t[n]
    minT_duration = m * timestep
    Tm_start = n-math.floor((m-1)/2)
    Tm_end = Tm_start + m
    Tm_depth = flux[Tm_start:Tm_end].mean() 
    Ts = nonzero(T1[m]).std() # only the box width selected. Not RMS of all T-statistic
    
    asym, width1, width2, depth, info = calc_shape(m,n,t,flux,quality,flux_error,n_m_bg_start=2)
    return info[0], info[1], info[3]

def plot(data):
    time, flux, error = search_method(data)
    fig, ax = plt.subplots(1,2,figsize=(15,7))
    try:
        ax[0].plot(data["time"],data["corrected flux"])
    except:
        ax[0].plot(data["TIME"],data["PDCSAP_FLUX"])
    ax[1].plot(time,flux)
    ax[1].plot(time,fit,label='gaussian')
    ax[1].plot(time,fit2,color='black',label='skewed gaussian')
    ax[1].legend()
    #ax[1].errorbar(time,flux, yerr=error, fmt='', color='b')
    plt.show()
    
def calc_reduced_chi_square(fit, x, y, yerr, N, n_free):
    '''
    fit (array) values for the fit
    x,y,yerr (arrays) data
    N total number of points
    n_free number of parameters we are fitting
    '''
    return 1.0/(N-n_free)*sum(((fit - y)/yerr)**2)

def normalise_error(flux, flux_error):
    return flux_error/np.nanmedian(flux)

In [ ]:
lookup = pd.read_csv("/storage/astro2/phrdhx/tesslcs/sector1lookup.csv")

## Example 1

In [ ]:
data, lc_info = import_lightcurve("betapic/test/hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits")

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)

### 1a. Flip this lightcurve to see if the skew works in reverse as well

In [ ]:
data = data.to_pandas()

In [ ]:
data.head()

In [ ]:
data.PDCSAP_FLUX = data.PDCSAP_FLUX.values[::-1]

In [ ]:
data = Table.from_pandas(data)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

Appears to fit the data quite well.

### Goodness of fit of model

In [ ]:
calc_reduced_chi_square(fit2,time,flux,error,len(flux),4)

In [ ]:
#chisquare(flux,fit2)

---

## Example 2 (CTOI)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/2_min_cadence_targets/tesslc_50380257.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)

---
## Example 3 (CTOI)

In [ ]:
lookup_tic(55659311)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/2_min_cadence_targets/tesslc_55659311.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
fit

In [ ]:
plot(data)

In [ ]:
plt.scatter(time,flux)

In [ ]:
len(flux)

---
## Example 4 (EA2)

In [ ]:
lookup_tic(60646247)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/tesslcs_tmag_11_12/tesslc_60646247.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
np.diag(cov1)

In [ ]:
np.sqrt((cov1)[0,0])

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)

---
## Example 5 (WASP 100)

In [ ]:
lookup_tic(38846515)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/2_min_cadence_targets/tesslc_38846515.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
params

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)

---
## Example 6

In [ ]:
lookup_tic(61178236)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/tesslcs_tmag_10_11/tesslc_61178236.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
params

In [ ]:
params1

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)

### Test interpolate

In [ ]:
arange = np.linspace(time.min(),time.max(),5000)
interp = np.interp(arange,time,flux)

In [ ]:
plt.scatter(arange,interp)

## Example 7

In [ ]:
lookup_tic(441422220)

In [ ]:
data, lc_info = import_XRPlightcurve("/storage/astro2/phrdhx/tesslcs/tesslcs_sector_1_104/2_min_cadence_targets/tesslc_441422220.pkl",sector=1)

In [ ]:
time, flux, error = search_method(data)

In [ ]:
params, cov = single_gaussian_curve_fit(time,-flux)
params1, cov1 = skewed_gaussian_curve_fit(time,flux)

In [ ]:
print(f"skewness is {round(params1[0],3)}+/-{np.sqrt(np.diag(cov1)[0])}")

In [ ]:
params

In [ ]:
params1

In [ ]:
fit = -gauss(time,*params)
fit2 = updated_skewed_gauss(time,*params1)

In [ ]:
plot(data)

In [ ]:
len(flux)